In [3]:
import tokenizer
import pickle
import pandas as pd
import numpy as np
import glob
import os
import string
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams
import json
import sparse_matrix_functions
import scipy.sparse as scs

In [4]:
path = '/Users/ale/Dropbox (Yadlin Family)/galvanize/capstone/*.txt'

In [ ]:
with open("gene_dictionary_final.pickle", "rb") as dict_gene:
        gene_dict = pickle.load(dict_gene)

In [ ]:
gene_keys = set(gene_dict.keys())
gene_values = set(gene_dict.values())

In [ ]:
gene_dict['her2']

In [ ]:
with open("drug_dictionary_final.pickle", "rb") as dict_drug:
        drug_dict = pickle.load(dict_drug)

In [ ]:
drug_keys = set(drug_dict.keys())
drug_values = set(drug_dict.values())

In [ ]:
'#doxorubicin hydrochloride#' in drug_values

In [ ]:
drug_dict['acridine carboxamide']

In [ ]:
with open("greek_alphabet.pickle", "rb") as dict_greek:
        greek_dict = pickle.load(dict_greek)

In [5]:
import data_frame_creator
import sparse_matrix_functions

In [6]:
test=data_frame_creator.sparse_create_data_frame(path, short_list=True, min_df=0)

In [7]:
vocab_matrix =test[0]
vocab_matrix[vocab_matrix>1]=1
vocab_columns = test[1]
doc_names = test[2] 
orig_sentences =test[3]

In [8]:
col_idx=sparse_matrix_functions.column_indexing(vocab_columns)

In [58]:
vocab_columns.index('sensit')

1581

In [71]:
xl =vocab_matrix[:,vocab_columns.index('sensit')]

In [68]:
vocab_matrix[xl.todense().nonzero(),:]

IndexError: index returns 3-dim structure

In [82]:
def select_rows(X,kind='A'):
    if kind == 'r' or kind =='R':
        ind =vocab_matrix[:,vocab_columns.index('resist')]
        X=X[ind.nonzero()[0],:]
    elif kind == 's' or kind =='S':
        ind =vocab_matrix[:,vocab_columns.index('sensit')]
        X=X[ind.nonzero()[0],:]
    else:
        X==X
    return X

        
        
        
        

In [112]:
def pick_network_type(X,kind):
    if kind == 'r' or kind =='R':
        Y=select_rows(X,'r')
        Z=sparse_matrix_functions.drop_columns(Y, [vocab_columns.index('sensit')])
    elif kind == 's' or kind =='S':
        Y=select_rows(X,'s')
        Z=sparse_matrix_functions.drop_columns(Y, [vocab_columns.index('resist')])
    else:
        X==sparse_matrix_functions.drop_columns(X,[vocab_columns.index('resist'),vocab_columns.index('sensit')])
        Z=scs.hstack((X,np.ones((X.shape[0],1))[:,None])).A
    return Z

In [113]:
vocab_matrix

<2894x1582 sparse matrix of type '<class 'numpy.int64'>'
	with 4224 stored elements in Compressed Sparse Row format>

In [114]:
test = pick_network_type(vocab_matrix,'r')

In [115]:
test

<136x1581 sparse matrix of type '<class 'numpy.int64'>'
	with 1720 stored elements in Compressed Sparse Row format>

In [116]:
test_2=pick_network_type(vocab_matrix,'a')

/Users/ale/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:225: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  " != instead.", SparseEfficiencyWarning)


TypeError: expected dimension <= 2 array or matrix

In [106]:
np.ones((1500,1))

array([[ 1.],
       [ 1.],
       [ 1.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.]])

In [ ]:
non_empty_rows = sparse_matrix_functions.non_zero_rows(vocab_matrix)

In [ ]:
sentences = [orig_sentences[i] for i in non_empty_rows]

In [ ]:
files = [doc_names[i] for i in non_empty_rows]

In [ ]:
vocab_mat_clean=sparse_matrix_functions.remove_zero_rows(vocab_matrix)

In [ ]:
vocab_mat_clean

In [100]:
vocab_matrix.shape

(2894, 1582)

In [130]:
print(list(vocab_matrix[:,vocab_columns.index('sensit')].indices))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [104]:
sensit2

<2894x1 sparse matrix of type '<class 'numpy.int64'>'
	with 47 stored elements in Compressed Sparse Row format>

In [89]:
network_genes=sparse_matrix_functions.get_network_rows(vocab_columns)

In [90]:
network_drugs =sparse_matrix_functions.get_network_columns(vocab_columns)

In [7]:
resist = sparse_matrix_functions.drop_columns(vocab_matrix, [vocab_columns.index('sensit')])

In [8]:
sensit = sparse_matrix_functions.drop_columns(vocab_matrix, [vocab_columns.index('resist')])

In [11]:
sensit_network_matrix = sparse_matrix_functions.make_network_matrix(sensit, vocab_columns)

In [12]:
resist_network_matrix = sparse_matrix_functions.make_network_matrix(resist, vocab_columns)

In [13]:
pairs, counts = sparse_matrix_functions.extract_gene_drug_pairs(sensit_network_matrix, network_genes, network_drugs)

In [20]:
original_indeces = sparse_matrix_functions.back_to_original_indeces(pairs, col_idx)

In [89]:
sensit_evidence_akt_lapatinib = sparse_matrix_functions.get_evidence_sentences('akt','lapatinib','r',7, test, original_indeces)

In [90]:
sensit_evidence_akt_lapatinib

['almost_vocab_drugs.txt: sentence too long to display',
 'test.txt: Treatment of the lapatinib-resistant HER2+ cells with Src inhibitors suppressed the PI3K PTEN Akt mTOR pathway as well as  Figure 5: Sites of Targeting the EGFR PI3K PTEN Akt mTORC Pathway with Small Molecule Membrane-Permeable Inhibitors and Monoclonal Antibodies (MoAbs).']

In [46]:
pairs2, counts2 = sparse_matrix_functions.extract_gene_drug_pairs(resist_network_matrix, network_genes, network_drugs)

In [47]:
original_indeces2 = sparse_matrix_functions.back_to_original_indeces(pairs2, col_idx)

In [82]:
resist_evidence_akt_lapatinib = sparse_matrix_functions.get_evidence_sentences('akt','lapatinib','r',7, 
                                                                               test, original_indeces2)

In [83]:
resist_evidence_akt_lapatinib

['almost_vocab_drugs.txt: sentence too long to display',
 'test.txt: Treatment of the lapatinib-resistant HER2+ cells with Src inhibitors suppressed the PI3K PTEN Akt mTOR pathway as well as  Figure 5: Sites of Targeting the EGFR PI3K PTEN Akt mTORC Pathway with Small Molecule Membrane-Permeable Inhibitors and Monoclonal Antibodies (MoAbs).']